In [225]:
import pickle
import pandas as pd
import torch
import torch.nn as nn
import torch
import networkx as nx
from torch_geometric.data import Data
from sklearn.metrics import roc_auc_score
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

In [198]:
with open('../Data/MTT_graphs/trade_graphs.pkl', 'rb') as f:
    trade_graphs = pickle.load(f)

single_trade_graph = trade_graphs[-1]

In [199]:
trade_df = pd.DataFrame(single_trade_graph.edges(data=True), columns=['Source', 'Target', 'Attributes'])
trade_df = pd.concat([trade_df.drop(['Attributes'], axis=1), trade_df['Attributes'].apply(pd.Series)], axis=1)

In [200]:
with open('../all_wb_indicators.pickle', 'rb') as f:
    wb_indicators = pickle.load(f)

In [ ]:
wb_year_dict = {}

count = 0
for feature, df in wb_indicators.items():

    if count % 100 == 0:
        print(count)
    count += 1
    
    df = df.set_index('economy')
    df_transposed = df.transpose()
    
    for year in df_transposed.index:
        if year not in wb_year_dict:
            wb_year_dict[year] = pd.DataFrame(df_transposed.loc[year]).rename(columns={year: feature})
        else:
            wb_year_dict[year] = pd.concat([wb_year_dict[year], pd.DataFrame(df_transposed.loc[year]).rename(columns={year: feature})], axis=1)


In [ ]:
#with open('../Data/wb_features.pkl', 'wb') as f:
#    pickle.dump(wb_year_dict, f)

In [201]:
with open('../Data/wb_features.pkl', 'rb') as f:
    wb_year_dict = pickle.load(f)

wb_2018 = wb_year_dict['YR2018']

In [202]:
wb_2018_indices = set(wb_2018.index)
trade_df = trade_df[trade_df['Source'].isin(wb_2018_indices) & trade_df['Target'].isin(wb_2018_indices)]

In [203]:
nodes = pd.concat([trade_df['Source'], trade_df['Target']]).unique()
node_to_int = {node: i for i, node in enumerate(nodes)}

trade_df['source_int'] = trade_df['Source'].map(node_to_int)
trade_df['target_int'] = trade_df['Target'].map(node_to_int)

In [204]:
# Create a set of unique nodes in trade_df
trade_nodes = set(pd.concat([trade_df['Source'], trade_df['Target']]).unique())
wb_2018 = wb_2018[wb_2018.index.isin(trade_nodes)]

In [205]:
wb_2018 = wb_2018.fillna(0)

In [206]:
variance = wb_2018.var()
low_variance_cols = variance[variance < 0.1].index

In [207]:
wb_2018 = wb_2018.drop(columns=low_variance_cols)

In [208]:
# Create a mapping of node names to integers
nodes = pd.concat([trade_df['Source'], trade_df['Target']]).unique()
node_to_int = {node: i for i, node in enumerate(nodes)}

# Add new columns to trade_df
trade_df['source_int'] = trade_df['Source'].map(node_to_int)
trade_df['target_int'] = trade_df['Target'].map(node_to_int)

# Replace the index of wb_2018
wb_2018.index = wb_2018.index.map(node_to_int)

In [209]:
edge_index = torch.tensor(trade_df[['source_int', 'target_int']].values, dtype=torch.long).t().contiguous()
node_features = torch.tensor(wb_2018.values, dtype=torch.float)
data = Data(x=node_features, edge_index=edge_index)

In [210]:
data = T.NormalizeFeatures()(data)

In [218]:
split = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    is_undirected=True,
    add_negative_train_samples=False,
    neg_sampling_ratio=2.0,
)
train_data, val_data, test_data = split(data)

In [230]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = nn.Dropout(0.5)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [231]:
model = Net(data.num_features, 64, 32)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [232]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [233]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Epoch: 001, Loss: 0.6931, Val: 0.5774, Test: 0.5736
Epoch: 002, Loss: 0.6910, Val: 0.5774, Test: 0.5736
Epoch: 003, Loss: 0.6865, Val: 0.5773, Test: 0.5736
Epoch: 004, Loss: 0.6872, Val: 0.5774, Test: 0.5736
Epoch: 005, Loss: 0.6852, Val: 0.5774, Test: 0.5736
Epoch: 006, Loss: 0.6847, Val: 0.5774, Test: 0.5736
Epoch: 007, Loss: 0.6852, Val: 0.5774, Test: 0.5736
Epoch: 008, Loss: 0.6852, Val: 0.5774, Test: 0.5736
Epoch: 009, Loss: 0.6849, Val: 0.5774, Test: 0.5736
Epoch: 010, Loss: 0.6841, Val: 0.5774, Test: 0.5736
Epoch: 011, Loss: 0.6841, Val: 0.5774, Test: 0.5736
Epoch: 012, Loss: 0.6840, Val: 0.5774, Test: 0.5736
Epoch: 013, Loss: 0.6840, Val: 0.5774, Test: 0.5736
Epoch: 014, Loss: 0.6839, Val: 0.5774, Test: 0.5736
Epoch: 015, Loss: 0.6837, Val: 0.5774, Test: 0.5736
Epoch: 016, Loss: 0.6836, Val: 0.5774, Test: 0.5736
Epoch: 017, Loss: 0.6831, Val: 0.5774, Test: 0.5736
Epoch: 018, Loss: 0.6828, Val: 0.5774, Test: 0.5736
Epoch: 019, Loss: 0.6827, Val: 0.5774, Test: 0.5736
Epoch: 020, 

In [196]:
data

Data(x=[192, 1272], edge_index=[2, 16222])